In [ ]:
import os
%load_ext autoreload
%autoreload 2
# Get current working directory
cwd = os.getcwd()
print(cwd)


/sfs/gpfs/tardis/project/SDS/capstones_tashman/tumor_control/data/b/deep l


### EfficientNet Model

Training Time: 2471 sec 

87% Accuracy

In [9]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Dataset, WeightedRandomSampler
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms.functional import to_pil_image, normalize 
from tqdm import tqdm
import kagglehub
from PIL import Image, UnidentifiedImageError
from sklearn.metrics import confusion_matrix, classification_report
import subprocess

# Set random seeds for reproducibility
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
setup_seed(SEED)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

BATCH_SIZE = 32
TEST_RATIO = 0.15
VAL_RATIO = 0.15 
NUM_EPOCHS = 30  
LEARNING_RATE = 3e-4
PATIENCE = 7
WEIGHT_DECAY = 1e-4
MOMENTUM = 0.9
NUM_WORKERS = 2 
UNFREEZE_EPOCH = 5 
MIXUP_ALPHA = 0.2 
LABEL_SMOOTHING = 0.1 

try:
    import pkg_resources
    try:
        pkg_resources.get_distribution('pillow-webp-fork') 
        print("pillow-webp-fork already installed.")
    except pkg_resources.DistributionNotFound:
        print("Attempting to install pillow-webp-fork...")
        subprocess.check_call(["pip", "install", "pillow-webp-fork"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("Successfully installed pillow-webp-fork.")
        print("Please restart the kernel/script if WEBP images still cause errors.")
except Exception as e:
    print(f"Could not install WebP support: {e}")
    print("WEBP images might not load correctly.")

# Download dataset from Kaggle
print("Downloading dataset from Kaggle...")
dataset_path_root = kagglehub.dataset_download("sumn2u/garbage-classification-v2")
print(f"Dataset downloaded to: {dataset_path_root}")
possible_data_folders = ["dataset", "garbage-dataset", "Garbage Classification V2"]

data_dir = None
if os.path.exists(os.path.join(dataset_path_root, "dataset")):
     data_dir = os.path.join(dataset_path_root, "dataset")
elif os.path.exists(os.path.join(dataset_path_root, "garbage-dataset")):
     data_dir = os.path.join(dataset_path_root, "garbage-dataset")
else:
    for folder_name in possible_data_folders:
        potential_dir = os.path.join(dataset_path_root, folder_name)
        if os.path.isdir(potential_dir):
            subdirs = [d for d in os.listdir(potential_dir) if os.path.isdir(os.path.join(potential_dir, d))]
            if any(cls_name in subdirs for cls_name in ['cardboard', 'glass', 'plastic', 'trash']):
                 data_dir = potential_dir
                 break
    if data_dir is None:
        subdirs = [d for d in os.listdir(dataset_path_root) if os.path.isdir(os.path.join(dataset_path_root, d))]
        if len(subdirs) == 1:
             data_dir = os.path.join(dataset_path_root, subdirs[0])
        elif len(subdirs) > 1:
             for subdir in subdirs:
                 potential_dir = os.path.join(dataset_path_root, subdir)
                 sub_subdirs = [d for d in os.listdir(potential_dir) if os.path.isdir(os.path.join(potential_dir, d))]
                 if any(cls_name in sub_subdirs for cls_name in ['cardboard', 'glass', 'plastic', 'trash']):
                      data_dir = potential_dir
                      break
if data_dir is None:
    print(f"Warning: Could not reliably determine data directory. Assuming it's: {dataset_path_root}")
    data_dir = dataset_path_root
elif not os.path.exists(data_dir):
     print(f"Error: Determined data directory '{data_dir}' does not exist.")
     raise FileNotFoundError(f"Data directory '{data_dir}' not found.")

print(f"Using data directory: {data_dir}")


# create a robust dataset class that skips corrupted images
class RobustImageFolder(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.samples = []
        self.classes = []
        self.class_to_idx = {}
        self.targets = []

        if not os.path.isdir(root):
             raise FileNotFoundError(f"The specified root directory does not exist: {root}")

        try:
            potential_classes = sorted([d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))])
            if not potential_classes:
                raise ValueError(f"No subdirectories found in {root}. Check the data directory path.")
            self.classes = potential_classes
        except OSError as e:
             raise OSError(f"Cannot list directories in {root}. Check permissions or path.") from e

        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        class_counts = {cls: 0 for cls in self.classes}
        skipped_count = 0

        print("Scanning dataset for valid images...")
        for class_name in self.classes:
            class_dir = os.path.join(root, class_name)
            if not os.path.isdir(class_dir):
                print(f"Warning: Expected class directory not found: {class_dir}")
                continue

            class_idx = self.class_to_idx[class_name]
            try:
                filenames = os.listdir(class_dir)
            except OSError as e:
                print(f"Warning: Could not read files in {class_dir}: {e}")
                continue

            print(f"Processing class: {class_name} ({len(filenames)} files)")

            for filename in filenames:
                path = os.path.join(class_dir, filename)
                if not os.path.isfile(path):
                    continue 

                try:
                    with open(path, 'rb') as f:
                         img = Image.open(f)
                         img.verify() 

                    self.samples.append((path, class_idx))
                    self.targets.append(class_idx)
                    class_counts[class_name] += 1
                except (UnidentifiedImageError, IOError, OSError, SyntaxError, ValueError) as e:

                    skipped_count += 1
                except Exception as e: 
                    print(f"Unexpected error skipping image {path}: {type(e).__name__} - {e}")
                    skipped_count += 1

        if skipped_count > 0:
            print(f"Skipped {skipped_count} corrupt or unreadable images.")

        print("\nClass distribution (Valid images):")
        total_valid = 0
        for cls, count in class_counts.items():
            print(f"  {cls}: {count} images")
            total_valid += count

        if total_valid == 0:
             raise RuntimeError(f"No valid images found in {root}. Please check the dataset content and path.")

        self.targets = np.array(self.targets)
        print(f"\nFound {total_valid} valid images across {len(self.classes)} classes.")


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        path, target = self.samples[index]
        try:
            with open(path, 'rb') as f:
                 sample = Image.open(f).convert('RGB')
        except Exception as e:
            print(f"Error loading image {path} during getitem: {e}. Returning placeholder.")

            placeholder_tensor = torch.zeros((3, 224, 224), dtype=torch.float32)
            return placeholder_tensor, target

        if self.transform is not None:
            try:
                 sample = self.transform(sample)
            except Exception as e:
                 print(f"Error applying transform to image {path}: {e}. Returning placeholder.")
                 placeholder_tensor = torch.zeros((3, 224, 224), dtype=torch.float32)
                 return placeholder_tensor, target

        return sample, target

IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Enhanced data transformations
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224, scale=(0.7, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(40),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1), 
    transforms.RandomGrayscale(p=0.1), 
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15), scale=(0.85, 1.15)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.2), ratio=(0.3, 3.3)),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Use the robust dataset
try:
    dataset_all = RobustImageFolder(data_dir, transform=transform_test)
    class_names = dataset_all.classes
    print(f"Loaded {len(class_names)} classes: {class_names}")
    print(f"Total dataset size: {len(dataset_all)} valid images")
except (FileNotFoundError, ValueError, RuntimeError, OSError) as e:
    print(f"Fatal error loading dataset: {e}")
    raise 

# Calculate class weights for balanced sampling
def calculate_class_weights(targets):
    class_sample_count = np.bincount(targets)
    class_sample_count = np.maximum(class_sample_count, 1) 

    total_samples = len(targets)
    num_classes = len(class_sample_count)

    class_weights = total_samples / (class_sample_count.astype(np.float32) * num_classes)

    weights = [class_weights[class_idx] for class_idx in targets]
    return weights, class_weights

# Calculate weights
sample_weights, class_weights_values = calculate_class_weights(dataset_all.targets)
print("\nClass weights for balanced sampling:")
for i, cls in enumerate(class_names):
    print(f"  {cls} (Index {i}): {class_weights_values[i]:.4f}")

# Create weighted sampler
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

# Split dataset
size_all = len(dataset_all)
size_test = int(size_all * TEST_RATIO)
size_val = int(size_all * VAL_RATIO)
size_train = size_all - size_test - size_val

if size_train <= 0 or size_val <= 0 or size_test <= 0:
    raise ValueError("Calculated split sizes are invalid. Check ratios and dataset size.")

print(f"\nSplitting dataset:")
print(f"  Total: {size_all}")
print(f"  Train: {size_train}")
print(f"  Validation: {size_val}")
print(f"  Test: {size_test}")

generator = torch.Generator().manual_seed(SEED)
# Split using indices because RobustImageFolder applies a default transform
# We want to apply train transform only to the train split later
indices = list(range(size_all))
random.shuffle(indices)

train_indices = indices[:size_train]
val_indices = indices[size_train : size_train + size_val]
test_indices = indices[size_train + size_val :]

# Create subset datasets using indices
dataset_train_subset = torch.utils.data.Subset(dataset_all, train_indices)
dataset_val_subset = torch.utils.data.Subset(dataset_all, val_indices)
dataset_test_subset = torch.utils.data.Subset(dataset_all, test_indices)

class TransformedSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.classes = self.subset.dataset.classes
        self.class_to_idx = self.subset.dataset.class_to_idx

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        # Get the data (which already had the test transform applied by RobustImageFolder)
        image_tensor, label = self.subset[idx]

        if self.transform:
            try:
                 # Denormalize first before converting to PIL
                 inv_normalize = transforms.Normalize(
                     mean=[-m/s for m, s in zip(IMAGENET_MEAN, IMAGENET_STD)],
                     std=[1/s for s in IMAGENET_STD]
                 )
                 image_tensor_denorm = inv_normalize(image_tensor)
                 image_pil = to_pil_image(image_tensor_denorm.cpu())
                 image = self.transform(image_pil) 
            except Exception as e:
                 print(f"Error applying specific transform in TransformedSubset for index {idx}: {e}. Returning original tensor.")
                 image = image_tensor
        else:
             image = image_tensor

        return image, label

# Apply the correct transforms using the wrapper
train_dataset = TransformedSubset(dataset_train_subset, transform=transform_train)
val_dataset = TransformedSubset(dataset_val_subset, transform=None) 
test_dataset = TransformedSubset(dataset_test_subset, transform=None) 

# Create DataLoaders
train_subset_targets = dataset_all.targets[train_indices]
train_subset_weights, _ = calculate_class_weights(train_subset_targets)
train_sampler = WeightedRandomSampler(
    weights=train_subset_weights,
    num_samples=len(train_subset_weights),
    replacement=True
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=train_sampler, 
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False,
    drop_last=True 
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False, 
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

def visualize_samples(dataloader, class_names, num_samples=16, filename='samples.png'):
    try:
        images, labels = next(iter(dataloader))
        num_to_show = min(num_samples, len(images))
        if num_to_show == 0:
             print("Warning: visualize_samples received an empty batch.")
             return

        images = images[:num_to_show]
        labels = labels[:num_to_show]

        n_rows = int(np.sqrt(num_to_show))
        n_cols = int(np.ceil(num_to_show / n_rows))

        plt.figure(figsize=(n_cols * 3, n_rows * 3))

        # Denormalize for display
        inv_normalize = transforms.Normalize(
            mean=[-m/s for m, s in zip(IMAGENET_MEAN, IMAGENET_STD)],
            std=[1/s for s in IMAGENET_STD]
        )

        for i in range(num_to_show):
            plt.subplot(n_rows, n_cols, i + 1)
            try:
                img_tensor = inv_normalize(images[i]) 
                img = to_pil_image(img_tensor.cpu()) 
                plt.imshow(img)
                plt.axis('off')
                # Ensure label index is within bounds
                if labels[i].item() < len(class_names):
                     plt.title(class_names[labels[i].item()], fontsize=10)
                else:
                     plt.title(f"Invalid Label: {labels[i].item()}", fontsize=10, color='red')

            except IndexError:
                 print(f"Error: Label index {labels[i].item()} out of range for class_names (len={len(class_names)})")
                 plt.imshow(np.ones((224, 224, 3)) * 0.5) # Gray placeholder
                 plt.axis('off')
                 plt.title("Label Error", fontsize=10)
            except Exception as e:
                print(f"Error visualizing sample {i}: {e}")
                plt.imshow(np.ones((224, 224, 3)) * 0.5) # Gray placeholder
                plt.axis('off')
                plt.title("Viz Error", fontsize=10)

        plt.subplots_adjust(wspace=0.2, hspace=0.4) # Adjust spacing
        plt.tight_layout()
        plt.savefig(filename, dpi=150) # Save with decent resolution
        plt.close()
        print(f"Saved sample visualization to {filename}")

    except StopIteration:
        print("Error: DataLoader is empty. Cannot visualize samples.")
    except Exception as e:
        print(f"An unexpected error occurred in visualize_samples: {e}")

# Visualize samples before training
print("\nVisualizing some training samples...")
visualize_samples(train_loader, class_names, filename='training_samples.png')

# Model definition 
class WasteClassifier(nn.Module):
    def __init__(self, num_classes, model_name='efficientnet_b0', dropout_rate=0.4):
        super(WasteClassifier, self).__init__()
        self.model_name = model_name

        if model_name == 'efficientnet_b0':
            weights = models.EfficientNet_B0_Weights.IMAGENET1K_V1
            self.backbone = models.efficientnet_b0(weights=weights)
            self.in_features = self.backbone.classifier[1].in_features
            # Replace the final layer with an Identity layer
            self.backbone.classifier = nn.Identity()
            # Get the appropriate preprocessing transform for this model
            self.preprocess = weights.transforms()
            print(f"Using EfficientNet-B0 with {self.in_features} input features to classifier.")

        elif model_name == 'resnet50':
            weights = models.ResNet50_Weights.IMAGENET1K_V2
            self.backbone = models.resnet50(weights=weights)
            self.in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.preprocess = weights.transforms()
            print(f"Using ResNet50 with {self.in_features} input features to classifier.")
        else:
            raise ValueError(f"Unsupported model name: {model_name}. Choose 'efficientnet_b0' or 'resnet50'.")

        # Custom classifier head with adjusted dropout
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(self.in_features), # Add BatchNorm before Dropout
            nn.Dropout(p=dropout_rate),
            nn.Linear(self.in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512), # Add another BatchNorm
            nn.Dropout(p=dropout_rate / 2), # Slightly less dropout deeper
            nn.Linear(512, num_classes)
        )

        # Initialize classifier weights
        self._init_weights(self.classifier)

    def _init_weights(self, module):
        for m in module.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                # Initialize BatchNorm parameters
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        features = self.backbone(x)
        output = self.classifier(features)
        return output

    def freeze_backbone(self):
        print("Freezing backbone layers...")
        for param in self.backbone.parameters():
            param.requires_grad = False
        # Ensure classifier is trainable
        for param in self.classifier.parameters():
            param.requires_grad = True
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f"Trainable parameters after freezing: {trainable_params:,}")

    def unfreeze_backbone(self, unfreeze_layers=0):
        """Unfreezes layers. unfreeze_layers=0 means unfreeze all."""
        if unfreeze_layers <= 0:
             print("Unfreezing all backbone layers...")
             for param in self.backbone.parameters():
                 param.requires_grad = True
        else:
            print(f"Unfreezing the last {unfreeze_layers} layers/blocks of the backbone...")

            named_params = list(self.backbone.named_parameters())
            total_layers = len(named_params)
            layers_to_unfreeze = named_params[max(0, total_layers - unfreeze_layers):]

            print(f"Total backbone params: {total_layers}. Unfreezing {len(layers_to_unfreeze)} parameter groups.")

            unfrozen_names = []
            for name, param in layers_to_unfreeze:
                param.requires_grad = True
                unfrozen_names.append(name)


        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.parameters())
        print(f"Trainable parameters after unfreezing: {trainable_params:,} / {total_params:,} ({trainable_params/total_params:.2%})")


# Mixup Augmentation Functions
def mixup_data(x, y, alpha=0.2):
    """Applies mixup augmentation."""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0 # No mixup

    batch_size = x.size(0)
    index = torch.randperm(batch_size, device=x.device)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Calculates loss for mixup."""
    loss_a = criterion(pred, y_a)
    loss_b = criterion(pred, y_b)
    return lam * loss_a + (1 - lam) * loss_b

# Training Function (Handles freezing/unfreezing internally)
def train_model(model, train_loader, val_loader, num_epochs=NUM_EPOCHS, learning_rate=LEARNING_RATE,
               patience=PATIENCE, use_mixup=True, mixup_alpha=MIXUP_ALPHA,
               label_smoothing=LABEL_SMOOTHING, weight_decay=WEIGHT_DECAY,
               unfreeze_epoch=UNFREEZE_EPOCH, unfreeze_layers=0): # unfreeze_layers=0 means unfreeze all

    model.to(device)

    # Loss function with label smoothing
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    # Start with backbone frozen
    model.freeze_backbone()
    is_frozen = True

    # Optimizer (initially only for classifier)
    optimizer = optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()), # Only pass trainable params
        lr=learning_rate,
        weight_decay=weight_decay
    )

    # Scheduler 
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=num_epochs, eta_min=learning_rate / 100 # Reduce LR towards the end
    )

    # Tracking
    history = {'train_losses': [], 'val_losses': [], 'train_accuracies': [], 'val_accuracies': []}
    best_val_loss = float('inf')
    best_val_acc = 0.0
    epochs_no_improve = 0
    start_epoch = 0 

    print(f"\n--- Starting Training ---")
    print(f"Total epochs: {num_epochs}, Initial LR: {learning_rate}, Patience: {patience}")
    print(f"Mixup: {use_mixup} (alpha={mixup_alpha}), Label Smoothing: {label_smoothing}")
    print(f"Backbone unfreeze epoch: {unfreeze_epoch}")

    for epoch in range(start_epoch, num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        # --- Unfreezing Logic ---
        if is_frozen and epoch >= unfreeze_epoch:
            print(f"\n--- Unfreezing backbone at epoch {epoch+1} ---")
            model.unfreeze_backbone(unfreeze_layers=unfreeze_layers)
            is_frozen = False

            # Recreate optimizer with potentially lower LR for fine-tuning all layers
            fine_tune_lr = learning_rate / 10
            print(f"Updating optimizer for fine-tuning. New LR: {fine_tune_lr}")
            optimizer = optim.AdamW(
                filter(lambda p: p.requires_grad, model.parameters()),
                lr=fine_tune_lr,
                weight_decay=weight_decay
            )

            # Reset scheduler for the remaining epochs with the new optimizer and LR
            # Adjust T_max for the remaining epochs
            remaining_epochs = num_epochs - epoch
            scheduler = optim.lr_scheduler.CosineAnnealingLR(
                 optimizer, T_max=remaining_epochs, eta_min=fine_tune_lr / 100
            )
            print(f"Scheduler reset. T_max={remaining_epochs}, eta_min={fine_tune_lr / 100:.2e}")


        # --- Training Phase ---
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        train_bar = tqdm(train_loader, desc="[Train]", leave=False)

        for images, labels in train_bar:
            try:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()

                # Apply Mixup
                if use_mixup and not is_frozen:
                    images, labels_a, labels_b, lam = mixup_data(images, labels, mixup_alpha)
                    outputs = model(images)
                    loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)

                loss.backward()

 

                optimizer.step()

                running_loss += loss.item() * images.size(0) # Weighted by batch size

                # Accuracy calculation
                _, predicted = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                if use_mixup and not is_frozen:
                     correct_train += (lam * (predicted == labels_a).sum().item() + \
                                      (1 - lam) * (predicted == labels_b).sum().item())
                else:
                    correct_train += (predicted == labels).sum().item()

                train_bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{100*correct_train/total_train:.2f}%")

            except Exception as e:
                print(f"\nError during training batch: {e}")
                print(f"Image shape: {images.shape if 'images' in locals() else 'N/A'}")
                continue 

        epoch_train_loss = running_loss / total_train if total_train > 0 else 0
        epoch_train_acc = 100 * correct_train / total_train if total_train > 0 else 0
        history['train_losses'].append(epoch_train_loss)
        history['train_accuracies'].append(epoch_train_acc)

        # --- Validation Phase ---
        model.eval()
        running_loss_val = 0.0
        correct_val = 0
        total_val = 0
        val_bar = tqdm(val_loader, desc="[Valid]", leave=False)

        with torch.no_grad():
            for images, labels in val_bar:
                try:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels) 

                    running_loss_val += loss.item() * images.size(0)
                    _, predicted = torch.max(outputs.data, 1)
                    total_val += labels.size(0)
                    correct_val += (predicted == labels).sum().item()

                    val_bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{100*correct_val/total_val:.2f}%")
                except Exception as e:
                    print(f"\nError during validation batch: {e}")
                    continue

        epoch_val_loss = running_loss_val / total_val if total_val > 0 else float('inf')
        epoch_val_acc = 100 * correct_val / total_val if total_val > 0 else 0
        history['val_losses'].append(epoch_val_loss)
        history['val_accuracies'].append(epoch_val_acc)

        print(f"  Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%")
        print(f"  Valid Loss: {epoch_val_loss:.4f}, Valid Acc: {epoch_val_acc:.2f}%")
        print(f"  LR: {optimizer.param_groups[0]['lr']:.2e}") # Display current learning rate

        # Update LR scheduler
        scheduler.step()

        # --- Checkpointing and Early Stopping ---
        if epoch_val_loss < best_val_loss:
            print(f"  Validation loss improved ({best_val_loss:.4f} -> {epoch_val_loss:.4f}). Saving best loss model...")
            best_val_loss = epoch_val_loss
            torch.save(model.state_dict(), "best_model_loss.pth")
            epochs_no_improve = 0 # Reset counter


        if epoch_val_acc > best_val_acc:
             print(f"  Validation accuracy improved ({best_val_acc:.2f}% -> {epoch_val_acc:.2f}%). Saving best accuracy model...")
             best_val_acc = epoch_val_acc
             torch.save(model.state_dict(), "best_model_acc.pth")

             if epoch_val_loss >= best_val_loss: # Check if loss didn't improve
                  epochs_no_improve = 0 # Reset counter as acc improved
        elif epoch_val_loss >= best_val_loss: # Only increment if neither loss improved nor acc improved
             epochs_no_improve += 1
             print(f"  No improvement in val loss or acc for {epochs_no_improve} epoch(s).")


        if epochs_no_improve >= patience:
            print(f"\nEarly stopping triggered after {epoch + 1} epochs due to no improvement for {patience} epochs.")
            break

    # --- End of Training ---
    print(f"\nTraining finished after {epoch + 1} epochs.")
    # Load the best model based on validation accuracy
    try:
        print("Loading best model based on validation accuracy...")
        model.load_state_dict(torch.load("best_model_acc.pth", map_location=device))
        print(f"Loaded best model with validation accuracy: {best_val_acc:.2f}%")
    except FileNotFoundError:
        print("Warning: best_model_acc.pth not found. Using the model from the last epoch.")
    except Exception as e:
        print(f"Warning: Could not load best accuracy model: {e}. Using the model from the last epoch.")

    history['epochs_trained'] = epoch + 1
    return model, history

def plot_training_results(history, filename='training_results.png'):
    epochs_trained = history.get('epochs_trained', len(history['train_losses']))
    if epochs_trained == 0:
        print("No training history to plot.")
        return

    epochs = range(1, epochs_trained + 1)

    plt.figure(figsize=(14, 6))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_losses'], 'bo-', label='Training Loss')
    plt.plot(epochs, history['val_losses'], 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.ylim(bottom=0) 

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_accuracies'], 'bo-', label='Training Accuracy')
    plt.plot(epochs, history['val_accuracies'], 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.ylim(0, 100) # Accuracy is between 0 and 100

    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()
    print(f"Saved training plot to {filename}")

# Testing function remains largely the same
def test_model(model, test_loader, class_names, criterion=None):
    model.to(device)
    model.eval()

    if criterion is None:
        criterion = nn.CrossEntropyLoss() # Use standard loss for testing

    test_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    test_bar = tqdm(test_loader, desc="[Test]", leave=False)
    with torch.no_grad():
        for images, labels in test_bar:
            try:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                test_loss += loss.item() * images.size(0) # Weighted loss
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                test_bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{100*correct/total:.2f}%")
            except Exception as e:
                print(f"\nError during testing batch: {e}")
                continue

    if total == 0:
        print("Error: No test samples were processed.")
        return 0, 0, [], []

    avg_test_loss = test_loss / total
    test_accuracy = 100 * correct / total

    print("\n--- Test Results ---")
    print(f"  Average Loss: {avg_test_loss:.4f}")
    print(f"  Accuracy: {test_accuracy:.2f}% ({correct}/{total})")

    # Classification Report and Confusion Matrix
    if len(all_preds) > 0 and len(all_labels) > 0:
        try:
            print("\nClassification Report:")
            report = classification_report(all_labels, all_preds, target_names=class_names, digits=3, zero_division=0)
            print(report)

            # Plot Confusion Matrix (Normalized)
            cm = confusion_matrix(all_labels, all_preds)
            cm_norm = cm.astype('float') / np.maximum(cm.sum(axis=1)[:, np.newaxis], 1e-9) # Avoid division by zero

            plt.figure(figsize=(12, 10))
            plt.imshow(cm_norm, interpolation='nearest', cmap=plt.cm.Blues, vmin=0, vmax=1)
            plt.title('Normalized Confusion Matrix')
            plt.colorbar()
            tick_marks = np.arange(len(class_names))
            plt.xticks(tick_marks, class_names, rotation=45, ha='right')
            plt.yticks(tick_marks, class_names)

            fmt = '.2f'
            thresh = cm_norm.max() / 2.
            for i in range(cm_norm.shape[0]):
                for j in range(cm_norm.shape[1]):
                    plt.text(j, i, format(cm_norm[i, j], fmt),
                             ha="center", va="center",
                             color="white" if cm_norm[i, j] > thresh else "black")

            plt.tight_layout()
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            plt.savefig('confusion_matrix_normalized.png', dpi=300)
            plt.close()
            print("Saved normalized confusion matrix to confusion_matrix_normalized.png")

            # Plot Confusion Matrix (Raw Counts)
            plt.figure(figsize=(12, 10))
            plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Oranges)
            plt.title('Confusion Matrix (Raw Counts)')
            plt.colorbar()
            plt.xticks(tick_marks, class_names, rotation=45, ha='right')
            plt.yticks(tick_marks, class_names)

            fmt = 'd' # Integer format
            thresh = cm.max() / 2.
            for i in range(cm.shape[0]):
                for j in range(cm.shape[1]):
                    plt.text(j, i, format(cm[i, j], fmt),
                             ha="center", va="center",
                             color="white" if cm[i, j] > thresh else "black")

            plt.tight_layout()
            plt.ylabel('True label')
            plt.xlabel('Predicted label')
            plt.savefig('confusion_matrix_raw.png', dpi=300)
            plt.close()
            print("Saved raw confusion matrix to confusion_matrix_raw.png")

        except ValueError as ve:
             print(f"Error generating classification report/matrix: {ve}")
             print("This might happen if some classes are missing from predictions or labels in the test set.")
        except Exception as e:
            print(f"An error occurred during confusion matrix generation: {e}")
    else:
        print("Not enough data processed to generate classification report or confusion matrix.")

    return avg_test_loss, test_accuracy, all_preds, all_labels


# --- Visualize Predictions Function (Corrected) ---
def visualize_predictions(dataloader, class_names, model, num_samples=16, filename='prediction_results.png'):
    model.eval()
    model.to(device)

    inv_normalize = transforms.Normalize(
        mean=[-m/s for m, s in zip(IMAGENET_MEAN, IMAGENET_STD)],
        std=[1/s for s in IMAGENET_STD]
    )

    try:
        # Get a batch from the dataloader (test_loader is good for this)
        images, labels = next(iter(dataloader))
        images, labels = images.to(device), labels.to(device)

        num_to_show = min(num_samples, len(images))
        if num_to_show == 0:
            print("Warning: visualize_predictions received an empty batch.")
            return

        images_to_show = images[:num_to_show]
        labels_to_show = labels[:num_to_show]

        # Get predictions
        with torch.no_grad():
            outputs = model(images_to_show)
            _, predicted = torch.max(outputs, 1)

        n_rows = int(np.sqrt(num_to_show))
        n_cols = int(np.ceil(num_to_show / n_rows))

        plt.figure(figsize=(n_cols * 3.5, n_rows * 3.5)) # Slightly larger figure

        for i in range(num_to_show):
            plt.subplot(n_rows, n_cols, i + 1)
            try:
                img_tensor = inv_normalize(images_to_show[i]) # Denormalize
                img = to_pil_image(img_tensor.cpu()) # Convert tensor to PIL image
                plt.imshow(img)
                plt.axis('off')

                true_label = labels_to_show[i].item()
                pred_label = predicted[i].item()

                title_color = 'green' if pred_label == true_label else 'red'
                title = f"True: {class_names[true_label]}\nPred: {class_names[pred_label]}"
                plt.title(title, color=title_color, fontsize=10)
            except IndexError:
                 plt.imshow(np.ones((224, 224, 3)) * 0.5)
                 plt.axis('off')
                 plt.title("Label/Pred Error", fontsize=10, color='orange')
            except Exception as e:
                print(f"Error visualizing prediction for sample {i}: {e}")
                plt.imshow(np.ones((224, 224, 3)) * 0.5)
                plt.axis('off')
                plt.title("Viz Error", fontsize=10, color='red')

        plt.tight_layout()
        plt.savefig(filename, dpi=150)
        plt.close()
        print(f"Saved prediction visualization to {filename}")

    except StopIteration:
        print("Error: DataLoader is empty. Cannot visualize predictions.")
    except Exception as e:
        print(f"An unexpected error occurred in visualize_predictions: {e}")


# === Main Execution ===
if __name__ == "__main__":
    try:
        # Create the model (Using WasteClassifier as defined)
        model = WasteClassifier(
            num_classes=len(class_names),
            model_name='efficientnet_b0', 
            dropout_rate=0.4 # Adjust dropout if needed
        )
        print(f"Model '{model.model_name}' created.")

        # Train the model (handles freezing/unfreezing internally)
        model, history = train_model(
            model,
            train_loader,
            val_loader,
            num_epochs=NUM_EPOCHS,
            learning_rate=LEARNING_RATE,
            patience=PATIENCE,
            use_mixup=True, # Enable Mixup
            mixup_alpha=MIXUP_ALPHA,
            label_smoothing=LABEL_SMOOTHING,
            weight_decay=WEIGHT_DECAY,
            unfreeze_epoch=UNFREEZE_EPOCH, # Epoch to start fine-tuning backbone
            unfreeze_layers=0 # Unfreeze all layers when fine-tuning starts (0 means all)
        )

        # Plot training results
        print("\n--- Plotting Training Results ---")
        plot_training_results(history, filename='training_results.png')

        # Test the final model (best one loaded within train_model)
        print("\n--- Evaluating Model on Test Set ---")
        test_loss, test_accuracy, _, _ = test_model(model, test_loader, class_names)

        # Visualize predictions on a batch from the test set
        print("\n--- Visualizing Predictions on Test Set ---")
        # Use test_loader directly for visualization
        visualize_predictions(test_loader, class_names, model, filename='test_predictions.png')

        print("\n--- Execution Finished ---")
        print(f"Final Test Accuracy: {test_accuracy:.2f}%")
        print("Plots and model checkpoints ('best_model_acc.pth', 'best_model_loss.pth') saved.")

    except FileNotFoundError as e:
        print(f"\n--- Fatal Error: File Not Found ---")
        print(f"{e}")
        print("Please ensure the dataset path is correct and the files exist.")
    except ValueError as e:
        print(f"\n--- Fatal Error: Value Error ---")
        print(f"{e}")
        print("This might be due to incorrect configuration (e.g., split ratios) or dataset issues.")
    except RuntimeError as e:
         if "CUDA out of memory" in str(e):
              print(f"\n--- Fatal Error: CUDA Out of Memory ---")
              print("Try reducing BATCH_SIZE or using a smaller model.")
         else:
              print(f"\n--- Fatal Error: Runtime Error ---")
              print(f"{e}")
         import traceback
         traceback.print_exc()
    except Exception as e:
        print(f"\n--- An Unexpected Error Occurred ---")
        print(f"Error Type: {type(e).__name__}")
        print(f"Error Details: {e}")
        import traceback
        traceback.print_exc()

Using device: cuda
Attempting to install pillow-webp-fork...
Could not install WebP support: Command '['pip', 'install', 'pillow-webp-fork']' returned non-zero exit status 1.
WEBP images might not load correctly.
Dataset downloaded to: /home/gry6ks/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/8
Using data directory: /home/gry6ks/.cache/kagglehub/datasets/sumn2u/garbage-classification-v2/versions/8/garbage-dataset
Scanning dataset for valid images...
Processing class: battery (944 files)
Processing class: biological (997 files)
Processing class: cardboard (1825 files)
Processing class: clothes (5327 files)
Processing class: glass (3061 files)
Processing class: metal (1020 files)
Processing class: paper (1680 files)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:3296: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


Processing class: plastic (1984 files)
Processing class: shoes (1977 files)
Processing class: trash (947 files)
Skipped 3 corrupt or unreadable images.

Class distribution (Valid images):
  battery: 944 images
  biological: 997 images
  cardboard: 1825 images
  clothes: 5327 images
  glass: 3061 images
  metal: 1020 images
  paper: 1677 images
  plastic: 1984 images
  shoes: 1977 images
  trash: 947 images

Found 19759 valid images across 10 classes.
Loaded 10 classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
Total dataset size: 19759 valid images

Class weights for balanced sampling:
  battery (Index 0): 2.0931
  biological (Index 1): 1.9818
  cardboard (Index 2): 1.0827
  clothes (Index 3): 0.3709
  glass (Index 4): 0.6455
  metal (Index 5): 1.9372
  paper (Index 6): 1.1782
  plastic (Index 7): 0.9959
  shoes (Index 8): 0.9994
  trash (Index 9): 2.0865

Splitting dataset:
  Total: 19759
  Train: 13833
  Validation: 2963

  Train Loss: 9.2265, Train Acc: 39.18%
  Valid Loss: 4.3591, Valid Acc: 71.01%
  LR: 3.00e-04
  Validation loss improved (inf -> 4.3591). Saving best loss model...
  Validation accuracy improved (0.00% -> 71.01%). Saving best accuracy model...

Epoch 2/30


  Train Loss: 7.0080, Train Acc: 50.08%
  Valid Loss: 8.3847, Valid Acc: 74.38%
  LR: 2.99e-04
  Validation accuracy improved (71.01% -> 74.38%). Saving best accuracy model...

Epoch 3/30


...

  Train Loss: 3.5975, Train Acc: 56.16%
  Valid Loss: 1.4667, Valid Acc: 86.26%
  LR: 2.14e-06
  No improvement in val loss or acc for 3 epoch(s).

Epoch 28/30


  Train Loss: 3.5373, Train Acc: 56.27%
  Valid Loss: 1.5000, Valid Acc: 86.64%
  LR: 1.34e-06
  No improvement in val loss or acc for 4 epoch(s).

Epoch 29/30


  Train Loss: 3.5179, Train Acc: 56.28%
  Valid Loss: 1.4189, Valid Acc: 86.67%
  LR: 7.67e-07
  No improvement in val loss or acc for 5 epoch(s).

Epoch 30/30


  Train Loss: 3.6295, Train Acc: 54.76%
  Valid Loss: 1.4760, Valid Acc: 87.07%
  LR: 4.17e-07
  No improvement in val loss or acc for 6 epoch(s).

Training finished after 30 epochs.
Loading best model based on validation accuracy...
Loaded best model with validation accuracy: 87.24%

--- Plotting Training Results ---
Saved training plot to training_results.png

--- Evaluating Model on Test Set ---



--- Test Results ---
  Average Loss: 0.4883
  Accuracy: 87.14% (2582/2963)

Classification Report:
              precision    recall  f1-score   support

     battery      0.869     0.974     0.919       116
  biological      0.915     0.952     0.933       147
   cardboard      0.915     0.881     0.898       295
     clothes      0.979     0.950     0.964       780
       glass      0.966     0.727     0.830       469
       metal      0.531     0.969     0.686       161
       paper      0.897     0.798     0.844       262
     plastic      0.747     0.845     0.793       297
       shoes      0.954     0.909     0.931       297
       trash      0.721     0.727     0.724       139

    accuracy                          0.871      2963
   macro avg      0.849     0.873     0.852      2963
weighted avg      0.894     0.871     0.876      2963

Saved normalized confusion matrix to confusion_matrix_normalized.png
Saved raw confusion matrix to confusion_matrix_raw.png

--- Visualizing 